In [1]:
import glob
from os.path import join, expanduser

import xarray as xr
import dask
import datetime as dt

from dask.distributed import Client
from dask_jobqueue import PBSCluster

## Find your files:

In [2]:
sim_path = "/glade/campaign/cgd/cesm/CESM2-LE/atm/proc/tseries/day_1/" 
variable = "PRECT"

# example for one ensemble member: 
ensemble_member = "011"
sim_files = sorted(glob.glob(join(sim_path,variable,"b.e21.BHISTsmbb.f09_g17.LE2-1301."+ensemble_member+".*.nc")))

# How many files?
print (len(sim_files))
print("\n".join(sim_files))


17
/glade/campaign/cgd/cesm/CESM2-LE/atm/proc/tseries/day_1/PRECT/b.e21.BHISTsmbb.f09_g17.LE2-1301.011.cam.h1.PRECT.18500101-18591231.nc
/glade/campaign/cgd/cesm/CESM2-LE/atm/proc/tseries/day_1/PRECT/b.e21.BHISTsmbb.f09_g17.LE2-1301.011.cam.h1.PRECT.18600101-18691231.nc
/glade/campaign/cgd/cesm/CESM2-LE/atm/proc/tseries/day_1/PRECT/b.e21.BHISTsmbb.f09_g17.LE2-1301.011.cam.h1.PRECT.18700101-18791231.nc
/glade/campaign/cgd/cesm/CESM2-LE/atm/proc/tseries/day_1/PRECT/b.e21.BHISTsmbb.f09_g17.LE2-1301.011.cam.h1.PRECT.18800101-18891231.nc
/glade/campaign/cgd/cesm/CESM2-LE/atm/proc/tseries/day_1/PRECT/b.e21.BHISTsmbb.f09_g17.LE2-1301.011.cam.h1.PRECT.18900101-18991231.nc
/glade/campaign/cgd/cesm/CESM2-LE/atm/proc/tseries/day_1/PRECT/b.e21.BHISTsmbb.f09_g17.LE2-1301.011.cam.h1.PRECT.19000101-19091231.nc
/glade/campaign/cgd/cesm/CESM2-LE/atm/proc/tseries/day_1/PRECT/b.e21.BHISTsmbb.f09_g17.LE2-1301.011.cam.h1.PRECT.19100101-19191231.nc
/glade/campaign/cgd/cesm/CESM2-LE/atm/proc/tseries/day_1/PR

## Using Dask: 
Let's scale everything up using Dask.

In [3]:
# Create a PBS cluster object
cluster = PBSCluster(
    job_name = 'dask-wk23-hpc',
    cores = 1,
    memory = '4GiB',
    processes = 1,
    local_directory = '/local_scratch/pbs.$PBS_JOBID/dask/spill',
    resource_spec = 'select=1:ncpus=1:mem=4GB',
    queue = 'casper',
    walltime = '30:00',
    interface = 'ext'
)

In [4]:
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/negins/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/negins/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.106:42951,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/negins/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [5]:
num_workers = 32 
cluster.scale(num_workers)

client.wait_for_workers(num_workers)

In [14]:
!qstat -u $USER

                                                            Req'd  Req'd   Elap
Job ID          Username Queue    Jobname    SessID NDS TSK Memory Time  S Time
--------------- -------- -------- ---------- ------ --- --- ------ ----- - -----
2081278.casper* negins   htc      STDIN        8965   1   1    4gb 24:00 R 00:56
2081294.casper* negins   jhublog* cr-login-*  32756   1   1    4gb 720:0 R 00:53
2081757.casper* negins   htc      STDIN       71473   1   2   64gb 02:00 R 00:25
2082058.casper* negins   htc      dask-wk23*  28758   1   1    4gb 00:30 R 00:01
2082059.casper* negins   htc      dask-wk23*   5138   1   1    4gb 00:30 R 00:00
2082060.casper* negins   htc      dask-wk23*   5151   1   1    4gb 00:30 R 00:00
2082061.casper* negins   htc      dask-wk23*   5164   1   1    4gb 00:30 R 00:00
2082062.casper* negins   htc      dask-wk23*   5175   1   1    4gb 00:30 R 00:00
2082063.casper* negins   htc      dask-wk23*   5189   1   1    4gb 00:30 R 00:00
2082064.casper* negins   htc  

## Read in all Files: 

In [6]:
%%time
ds = xr.open_mfdataset(
    sorted(sim_files),
    # concatenate files in the order provided
    combine="by_coords",
    # parallelize the reading of individual files using dask
    # This means the returned arrays will be dask arrays
    parallel=True,
    # these are netCDF4 files, use the h5netcdf package to read them
    engine="h5netcdf",
    # hold off on decoding time
    decode_cf=False,
    # specify that data should be automatically chunked
    chunks="auto",
)
ds = xr.decode_cf(ds)


CPU times: user 1.86 s, sys: 313 ms, total: 2.18 s
Wall time: 20.9 s


In [7]:
ds

<xarray.Dataset> Size: 13GB
Dimensions:       (lat: 192, lon: 288, time: 60226, lev: 32, ilev: 33, nbnd: 2)
Coordinates:
  * lat           (lat) float64 2kB -90.0 -89.06 -88.12 ... 88.12 89.06 90.0
  * lon           (lon) float64 2kB 0.0 1.25 2.5 3.75 ... 356.2 357.5 358.8
  * lev           (lev) float64 256B 3.643 7.595 14.36 ... 957.5 976.3 992.6
  * ilev          (ilev) float64 264B 2.255 5.032 10.16 ... 967.5 985.1 1e+03
  * time          (time) object 482kB 1850-01-01 00:00:00 ... 2015-01-01 00:0...
Dimensions without coordinates: nbnd
Data variables: (12/26)
    gw            (time, lat) float64 93MB dask.array<chunksize=(3650, 192), meta=np.ndarray>
    hyam          (time, lev) float64 15MB dask.array<chunksize=(3650, 32), meta=np.ndarray>
    hybm          (time, lev) float64 15MB dask.array<chunksize=(3650, 32), meta=np.ndarray>
    P0            (time) float64 482kB ...
    hyai          (time, ilev) float64 16MB dask.array<chunksize=(3650, 33), meta=np.ndarray>
    hybi          (time, ilev) float64 16MB dask.array<chunksize=(3650, 33), meta=np.ndarray>
    ...            ...
    n2ovmr        (time) float64 482kB dask.array<chunksize=(3650,), meta=np.ndarray>
    f11vmr        (time) float64 482kB dask.array<chunksize=(3650,), meta=np.ndarray>
    f12vmr        (time) float64 482kB dask.array<chunksize=(3650,), meta=np.ndarray>
    sol_tsi       (time) float64 482kB dask.array<chunksize=(3650,), meta=np.ndarray>
    nsteph        (time) int32 241kB dask.array<chunksize=(3650,), meta=np.ndarray>
    PRECT         (time, lat, lon) float32 13GB dask.array<chunksize=(606, 192, 288), meta=np.ndarray>
Attributes:
    Conventions:       CF-1.0
    source:            CAM
    case:              b.e21.BHISTsmbb.f09_g17.LE2-1301.011
    logname:           sunseon
    host:              mom2
    initial_file:      b.e21.B1850.f09_g17.CMIP6-piControl.001.cam.i.1301-01-...
    topography_file:   /mnt/lustre/share/CESM/cesm_input/atm/cam/topo/fv_0.9x...
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    time_period_freq:  day_1

In [8]:
precip = ds.PRECT
precip

<xarray.DataArray 'PRECT' (time: 60226, lat: 192, lon: 288)> Size: 13GB
dask.array<concatenate, shape=(60226, 192, 288), dtype=float32, chunksize=(606, 192, 288), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 2kB -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon      (lon) float64 2kB 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time     (time) object 482kB 1850-01-01 00:00:00 ... 2015-01-01 00:00:00
Attributes:
    units:         m/s
    long_name:     Total (convective and large-scale) precipitation rate (liq...
    cell_methods:  time: mean

## Calculate Maximum Precip Per Year Per Ensemble: 

Note how the ouput dimensions (year: 166lat: 192lon: 288)

In [9]:
## calculate annual maximum: 

max_precip = precip.groupby(precip.time.dt.year, ).max()
max_precip

<xarray.DataArray 'PRECT' (year: 166, lat: 192, lon: 288)> Size: 37MB
dask.array<transpose, shape=(166, 192, 288), dtype=float32, chunksize=(3, 192, 288), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 2kB -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon      (lon) float64 2kB 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * year     (year) int64 1kB 1850 1851 1852 1853 1854 ... 2012 2013 2014 2015
Attributes:
    units:         m/s
    long_name:     Total (convective and large-scale) precipitation rate (liq...
    cell_methods:  time: mean

## Now you want to to save maximum Precip per year for each ensemble member: 

In [10]:
%%time
## check out the dashboard for compute heavy tasks: 
save_precip = max_precip.compute()

CPU times: user 2.13 s, sys: 455 ms, total: 2.59 s
Wall time: 54.8 s


In [11]:
output_file = "save_output_"+ensemble_member+".nc"
# Now, save the annual maxima time series for each ensemble member
save_precip.to_netcdf("b.e21.BHISTsmbb.f09_g17.LE2-1301.001.cam.h1.Rx1day.18500101-20141231.nc")